In [1]:
!pip install -U segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.2 MB/s eta 0:00:00


In [2]:
import torch
import segmentation_models_pytorch as smp
from torchvision import transforms
from PIL import Image
import numpy as np
import os
import time
import uuid

# ===================== DEVICE =====================

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===================== LOAD MODEL =====================

model = smp.DeepLabV3Plus(
    encoder_name="resnet101",
    encoder_weights=None,
    in_channels=3,
    classes=4
)

model.load_state_dict(
    torch.load("/kaggle/input/multi-segmentation/tensorflow2/default/1/best_multiclass_model (1).pth", map_location=DEVICE)
)
model.to(DEVICE)
model.eval()

print("🔥 Modèle chargé sur", DEVICE)

# ===================== PREPROCESS =====================

preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# ===================== COLOR MAP =====================

COLOR_MAP = np.array([
    [0, 0, 0],       # background
    [255, 0, 0],     # fibrin
    [0, 255, 0],     # granulation
    [0, 0, 255]      # callus
], dtype=np.uint8)

# ===================== ANALYSE MASK =====================

def analyze_mask(mask_array):
    h, w, _ = mask_array.shape
    total = h * w

    targets = {
        (255, 0, 0): "fibrin_red",
        (0, 255, 0): "granulation_green",
        (0, 0, 255): "callus_blue"
    }

    counts = {
        name: np.sum(np.all(mask_array == rgb, axis=-1))
        for rgb, name in targets.items()
    }

    percentages = {
        name: round((c / total) * 100, 2)
        for name, c in counts.items()
    }

    bg = total - sum(counts.values())
    percentages["background"] = round((bg / total) * 100, 2)

    return percentages

# ===================== MAIN FUNCTION =====================

def predict_image(image_path, save_mask=False, output_dir="masks"):
    """
    Predict segmentation mask + class percentages for ONE image
    """

    image_name = os.path.basename(image_path)

    # Load image
    img = Image.open(image_path).convert("RGB")

    # Preprocess
    x = preprocess(img).unsqueeze(0).to(DEVICE)

    # Inference
    with torch.no_grad():
        logits = model(x)
        pred = torch.argmax(logits, dim=1).cpu().numpy()[0]

    # Color mask
    color_mask = COLOR_MAP[pred]

    # Analysis
    stats = analyze_mask(color_mask)

    result = {
        "image": image_name,
        "analysis": stats
    }

    # Optional mask saving
    if save_mask:
        os.makedirs(output_dir, exist_ok=True)
        mask_name = f"mask_{uuid.uuid4().hex[:8]}.png"
        mask_path = os.path.join(output_dir, mask_name)
        Image.fromarray(color_mask).save(mask_path)
        result["mask_path"] = mask_path

    return result


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

🔥 Modèle chargé sur cuda


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

IMAGE_DIR = "/kaggle/input/wound-segmentation-images/data_wound_seg/train_images"
results = []

for img in tqdm(os.listdir(IMAGE_DIR)):
    if img.lower().endswith((".jpg", ".png", ".jpeg")):
        path = os.path.join(IMAGE_DIR, img)
        out = predict_image(path, save_mask=True)
        
        row = {
            "image": out["image"],
            **out["analysis"]
        }
        results.append(row)

df = pd.DataFrame(results)
df.to_csv("/kaggle/working/segmentation_results.csv", index=False)

df.head()


 36%|███▌      | 787/2208 [01:07<01:59, 11.85it/s]